In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from datetime import timedelta
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#데이터 불러오기
merge_df = pd.read_csv('/content/drive/MyDrive/미세먼지_농도_예측/data/csv/23~24_체감+날씨_홍.csv', encoding='cp949')
bodytemp_df = pd.read_csv('/content/drive/MyDrive/미세먼지_농도_예측/data/csv/(통합)25_체감온도_김.csv', encoding='cp949')
weather = pd.read_csv('/content/drive/MyDrive/미세먼지_농도_예측/data/csv/25_날씨(기상포털)_홍.csv', encoding='cp949')

In [ ]:
#컬럼명 변경
weather = weather.rename(columns={
    '지점': 'STN',
    '지점명': 'STN_NAME',
    '일시': 'TM',
    '평균기온(°C)': 'AVG_TA',
    '최저기온(°C)': 'MIN_TEMP',
    '최저기온 시각(hhmi)': 'MIN_TEMP_TIME',
    '최고기온(°C)': 'MAX_TEMP',
    '최고기온 시각(hhmi)': 'MAX_TEMP_TIME',
    '강수 계속시간(hr)': 'RAIN_DURATION',
    '10분 최다 강수량(mm)': 'MAX_RAIN_10M',
    '10분 최다강수량 시각(hhmi)': 'MAX_RAIN_10M_TIME',
    '1시간 최다강수량(mm)': 'MAX_RAIN_1H',
    '1시간 최다 강수량 시각(hhmi)': 'MAX_RAIN_1H_TIME',
    '일강수량(mm)': 'DAILY_RAIN',
    '최대 순간 풍속(m/s)': 'MAX_GUST_SPEED',
    '최대 순간 풍속 풍향(16방위)': 'MAX_GUST_DIR',
    '최대 순간풍속 시각(hhmi)': 'MAX_GUST_TIME',
    '최대 풍속(m/s)': 'MAX_WIND_SPEED',
    '최대 풍속 풍향(16방위)': 'MAX_WIND_DIR',
    '최대 풍속 시각(hhmi)': 'MAX_WIND_TIME',
    '평균 풍속(m/s)': 'AVG_WS',
    '풍정합(100m)': 'WIND_RUN',
    '최다풍향(16방위)': 'DOMINANT_WIND_DIR',
    '평균 이슬점온도(°C)': 'AVG_TD',
    '최소 상대습도(%)': 'MIN_HUMIDITY',
    '최소 상대습도 시각(hhmi)': 'MIN_HUMIDITY_TIME',
    '평균 상대습도(%)': 'AVG_RHM',
    '평균 증기압(hPa)': 'AVG_VAPOR_PRESSURE',
    '평균 현지기압(hPa)': 'AVG_LOCAL_PRESSURE',
    '최고 해면기압(hPa)': 'MAX_SEA_PRESSURE',
    '최고 해면기압 시각(hhmi)': 'MAX_SEA_PRESSURE_TIME',
    '최저 해면기압(hPa)': 'MIN_SEA_PRESSURE',
    '최저 해면기압 시각(hhmi)': 'MIN_SEA_PRESSURE_TIME',
    '평균 해면기압(hPa)': 'AVG_SEA_PRESSURE',
    '가조시간(hr)': 'SUNSHINE_DURATION',
    '합계 일조시간(hr)': 'TOTAL_SUNSHINE',
    '1시간 최다일사 시각(hhmi)': 'MAX_SOLAR_1H_TIME',
    '1시간 최다일사량(MJ/m2)': 'MAX_SOLAR_1H',
    '합계 일사량(MJ/m2)': 'TOTAL_SOLAR',
    '일 최심신적설(cm)': 'MAX_NEW_SNOW',
    '일 최심신적설 시각(hhmi)': 'MAX_NEW_SNOW_TIME',
    '일 최심적설(cm)': 'MAX_SNOW_DEPTH',
    '일 최심적설 시각(hhmi)': 'MAX_SNOW_DEPTH_TIME',
    '합계 3시간 신적설(cm)': 'NEW_SNOW_3H',
    '평균 전운량(1/10)': 'AVG_CLOUD_TOTAL',
    '평균 중하층운량(1/10)': 'AVG_CLOUD_LOW',
    '평균 지면온도(°C)': 'AVG_GROUND_TEMP',
    '최저 초상온도(°C)': 'MIN_SURFACE_TEMP',
    '평균 5cm 지중온도(°C)': 'SOIL_TEMP_5CM',
    '평균 10cm 지중온도(°C)': 'SOIL_TEMP_10CM',
    '평균 20cm 지중온도(°C)': 'SOIL_TEMP_20CM',
    '평균 30cm 지중온도(°C)': 'SOIL_TEMP_30CM',
    '0.5m 지중온도(°C)': 'SOIL_TEMP_50CM',
    '1.0m 지중온도(°C)': 'SOIL_TEMP_100CM',
    '1.5m 지중온도(°C)': 'SOIL_TEMP_150CM',
    '3.0m 지중온도(°C)': 'SOIL_TEMP_300CM',
    '5.0m 지중온도(°C)': 'SOIL_TEMP_500CM',
    '합계 대형증발량(mm)': 'EVAP_LARGE',
    '합계 소형증발량(mm)': 'EVAP_SMALL',
    '9-9강수(mm)': 'RAIN_9TO9',
    '기사': 'WEATHER_NOTE',
    '안개 계속시간(hr)': 'FOG_DURATION'
})

In [ ]:
# 학습데이터 칼럼지정
DATE_COL = 'TM'
TARGETS = ['TEMP']
test_feature = ['TMP','WSD','REH']# 'AVG_TD'
df = weather.merge(bodytemp_df, on="TM", how="inner")
df = df.dropna(subset=[DATE_COL] + TARGETS)
X_aligned = df[weather.columns]
y_aligned = df[bodytemp_df.columns]

In [ ]:
# 학습데이터 칼럼명 > api 통일
#컬럼명 변경
merge_df = merge_df.rename(columns={
    'AVG_TEMP': 'TMP',
    'AVG_WIND_SPEED': 'WSD',
    'AVG_DEW_POINT': 'AVG_TD',
    'AVG_HUMIDITY': 'REH',
})
#컬럼명 변경
weather = weather.rename(columns={
    'AVG_TA': 'TMP',
    'AVG_WS': 'WSD',
    'AVG_RHM': 'REH',
})

In [ ]:
TEST_merge_df = merge_df.dropna(subset=[DATE_COL] + TARGETS + test_feature)

In [ ]:
# 학습데이터 지정
X_train = TEST_merge_df[test_feature]
y_train = TEST_merge_df[TARGETS]

In [ ]:
# 데이터 학습
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# 모델 학습데이터 joblib 파일로 저장
import joblib

model_path = "/content/drive/MyDrive/미세먼지_농도_예측/data/joblib/체감온도예측이슬점X.joblib"
joblib.dump(model, model_path)
print("model saved:", model_path)

model saved: /content/drive/MyDrive/미세먼지_농도_예측/data/joblib/체감온도예측이슬점X.joblib


## api 불러오기

In [1]:
# SERVICE_KEY = "08a3cee75d1bbf834534a37b9b1a7967843c65220ba5f4e5c04dcacebab7a40b"
# X test 데이터 api 가져오기
base_date = datetime.now().strftime("%Y%m%d")
base_time = "0800"      # 3일치 안정
nx, ny = "55", "127"    # 서울 예시

SERVICE_KEY = tempkey
URL = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"

def get_fcst_df(base_date, base_time, nx, ny, num_rows=2000):
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": 1,
        "numOfRows": num_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny
    }

    items = requests.get(URL, params=params, timeout=20).json()["response"]["body"]["items"]["item"]
    df = pd.DataFrame(items)[["category", "fcstDate", "fcstTime", "fcstValue"]]

    df = (df.pivot_table(index=["fcstDate","fcstTime"], columns="category", values="fcstValue", aggfunc="first")
            .reset_index())

    df["datetime"] = pd.to_datetime(df["fcstDate"] + df["fcstTime"], format="%Y%m%d%H%M")
    return df.sort_values("datetime").reset_index(drop=True)

NameError: name 'datetime' is not defined

In [ ]:
# api 데이터 베이스로 변경
df_fcst = get_fcst_df(base_date, base_time, nx, ny)

In [ ]:
# X_test 데이터 지정
X_test= df_fcst[test_feature]

In [ ]:
# X_test 데이터 예측
y_pred = model.predict(X_test)